In [57]:

import numpy as np
import pandas as pd
import seaborn as sb
sb.set_style("dark")
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [58]:
# In Kaggle uncomment below line
# train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv')

# In Kaggle comment-out below line
train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv',  nrows=100)

mnist_train_data = train_df_org.loc[:, "pixel0":]
mnist_train_label = train_df_org.loc[:, "label"]

# test_df_org = np.genfromtxt('kaggle/input/digit-recognizer/test.csv', delimiter=',', skip_header=1)
# sample_submission_df = pd.read_csv('kaggle/input/digit-recognizer/sample_submission.csv')

In [59]:
# train_df_org.shape, test_df_org.shape, sample_submission_df.shape
train_df_org.shape

(100, 785)

In below the labels variable contains the labels and then from the 'train_df_org' (which contains all the actual image pixel) I will drop 'label' column

In [60]:
train_labels_org = train_df_org['label']
train_labels_org.shape


(100,)

In [61]:
train_df_org.drop(['label'], axis=1, inplace=True)
train_df_org.shape


(100, 784)

In [62]:
# Data-preprocessing: Standardizing the train dataframe
# (x - mean) / s.d

standardized_data = StandardScaler().fit_transform(train_df_org)
print(standardized_data.shape)

(100, 784)


#### Step-B => Build the Covariance-Matrix S which is $(X^T * X)$



In [63]:
# Now build the c-variance matrix which is : A^T * A
sample_data = standardized_data

# Matrix multiplication with numpy
covariance_matrix = np.matmul(sample_data.T, sample_data)

# As the sample_data has 784 columns, so the co-variance matrix shape
# should be 784 * 784
print('Shape of Co-variance matrix = ', covariance_matrix.shape)



Shape of Co-variance matrix =  (784, 784)


#### Step-C => Get the EigneVectors of this Covariance-Matrix S for projecting onto a 2-D space. These Eigenvectors will be sorted by the value of λ.

In [64]:
from scipy.linalg import eigh

# subset_by_index is an optional and iterable parameter
# If provided, this two-element iterable defines the start and the end indices
# of the desired eigenvalues (ascending order and 0-indexed).
# To return only the second smallest to fifth smallest eigenvalues, [1, 4] is used.
# [n-3, n-1] returns the largest three.
# Only available with “evr”, “evx”, and “gvx” drivers. The entries are directly converted to integers via int().
eigenvalues, eigenvectors = eigh(covariance_matrix, subset_by_index=(782, 783) )
print('Shape of Eigenvectors ', eigenvectors.shape )

Shape of Eigenvectors  (784, 2)


In [65]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import metrics
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Lambda, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D
from tensorflow.keras.optimizers import Adadelta
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler

In [66]:
nclasses = train_labels_org.max() - train_labels_org.min() + 1
mnist_train_label = to_categorical(train_labels_org, num_classes = nclasses)
print("Shape of ytrain after encoding: ", mnist_train_label.shape)


Shape of ytrain after encoding:  (100, 10)


In [67]:
def build_model(input_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))
    return model


def compile_model(model, optimizer='adam', loss='categorical_crossentropy'):
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


def train_model(model, train, test, epochs, split):
    history = model.fit(train, test, shuffle=True, epochs=epochs, validation_split=split)
    return history

In [68]:
cnn_model = build_model((28, 28, 1))
compile_model(cnn_model, 'adam', 'categorical_crossentropy')

# train the model for as many epochs as you want but I found training it above 80 will not help us and eventually increase overfitting.
model_history = train_model(cnn_model, train_df_org, train_labels_org, 80, 0.2)



Epoch 1/80


ValueError: in user code:

    /home/paul/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/paul/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/paul/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/paul/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/paul/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/paul/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/e